<a href="https://colab.research.google.com/github/omargfh/utils/blob/main/openai-meeting-summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SpeechRecognition
!pip install pydub
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.8 MB/s eta 0:00:00


In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
import os
import openai

In [71]:
error = lambda msg: print(f"\033[91m Error:\t\t \033[0m{msg}")
info = lambda msg: print(f"\033[94m INFO:\t\t \033[0m{msg}")
success = lambda msg: print(f"\033[92m Success:\t\t \033[0m{msg}")
warning = lambda msg: print(f"\033[93m Warning:\t\t \033[0m{msg}")

In [39]:
# https://blog.devgenius.io/how-to-transcribe-a-video-with-97-accuracy-using-python-f59bbf71d640
def transcribe(filepath: str):
  try:
    # Load the video file
    info("Converting video file")
    video = AudioSegment.from_file(filepath, format=filepath.split('.')[-1])
    audio = video.set_channels(1).set_frame_rate(16000).set_sample_width(2)
    audio.export("audio.wav", format="wav")
    success("Video file converted")
  except Exception as e:
    error(e)
    raise e

  # Initialize recognizer class (for recognizing the speech)
  try:
    info("Initializing Recoginzer")
    r = sr.Recognizer()
    success("Recognizer Initialized")
  except Exception as e:
    error(e)
    raise e

  # Open the audio file
  try:
    info("Reading audio file")
    with sr.AudioFile("audio.wav") as source:
      audio_text = r.record(source)
    success("Audio File Read")
  except Exception as e:
    error(e)
    raise e

  # Recognize the speech in the audio
  try:
    info("Recognizing Audio")
    text = r.recognize_google(audio_text, language='en-US')
    success("Audio Recognized")
  except Exception as e:
    error(e)
    raise e

  success("Transcription Complete")

  # return transcription
  return text

In [69]:
def summarize(text: str):
  def split_text(text):
    max_chunk_size = 2048
    chunks = []
    current_chunk = ""
    for sentence in text.split("."):
        if len(current_chunk) + len(sentence) < max_chunk_size:
            current_chunk += sentence + "."
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + "."
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

  info("Generating chunks")
  chunks = split_text(text)
  success("Chunks generated")

  def _generate_response_sync(prompt):
    response = openai.ChatCompletion.create( # Change this
        model = "gpt-3.5-turbo", # Change this
        messages = [ # Change this
            {"role": "user", "content": prompt}
        ],
        max_tokens = 1024,
        temperature = 0.5,
        stop=None
    )
    return response.choices[0].message.content

  def generate_summary(input_chunks):
    output_chunks = []
    for idx, chunk in enumerate(input_chunks):
        main_prompt = f"Please summarize the following meeting transcript and identify the different speakers:\n{chunk}\n\nSummary:"
        alt_prompt = f"Please summarize this portion of a meeting and identify the different speakers:\n{chunk}\n\nSummary:"
        response = _generate_response_sync(
            main_prompt if idx == 0 else alt_prompt
        )
        summary = response.strip()
        output_chunks.append(summary)
    return " ".join(output_chunks)

  info("Generating summaries")
  return generate_summary(chunks)


In [55]:
def generate_notes(filepath: str, api_key: str):
  openai.api_key = api_key
  transcript = transcribe(filepath)
  with open("transcript.txt", "w") as f:
    f.write(transcript)
  return summarize(transcript)

In [73]:
def execute(filepath: str, api_key: str):
  info("Starting...")
  notes = generate_notes(
      filepath,
      api_key
  )
  success("Notes generated")
  with open("notes.txt", "w") as f:
    info("Writing to notes.txt")
    f.write(notes)
    success("Done...")